### 1. Load data from file

In [0]:
import numpy as np

DATA_PATH = './data.txt'

# load data from file
data = np.genfromtxt(DATA_PATH, delimiter=',')

# separate features from labels
X_data = data[:, 0:-1]
Y_data = data[:, -1].reshape(-1, 1)

m = len(X_data)

### 2. Define functions for logistic regression



In [0]:
def logistic(theta, X):
  temp_x = np.concatenate((np.ones((m, 1)), X), axis=1)
  z = np.matmul(temp_x, theta.T)
  Y_hat = 1 / (1 + np.exp(-z))
  return Y_hat

def objective(Y_hat, Y):
  epsilon = 1e-8
  return (-1 / m) * np.sum(
      Y * np.log(Y_hat + epsilon) + (1 - Y) * np.log(1 - Y_hat + epsilon), axis=0
  )

def gradient(X, Y_hat, Y):
  temp_x = np.concatenate((np.ones((m, 1)), X), axis=1)
  return (1 / m) * np.matmul((Y_hat - Y).T, temp_x)

### 3. Learning with the gradient descent algorithm


In [0]:
theta = np.zeros((1, X_data.shape[1] + 1))
lr = 0.001
epoch_count = 20000

history = {
    'theta': np.zeros((epoch_count, theta.shape[1])),
    'train_err': np.zeros(epoch_count)
}

for epoch in range(epoch_count):
  # calculate training error
  Y_hat = logistic(theta, X_data)
  train_err = objective(Y_hat, Y_data)

  # log history
  history['theta'][epoch] = np.squeeze(theta)
  history['train_err'][epoch] = train_err

  # gradient descent
  grad_theta = gradient(X_data, Y_hat, Y_data)
  theta -= lr * grad_theta